In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
from PIL import Image
import numpy as np
import random
import glob

def adjust_image(data):
    imgs = tf.image.resize_images(data, (299, 299))
    return imgs

X=[]
X_processed=[]
Y=[]


classes = {}

path = "tumor_normal_patch/"
#converted_files = "/home/amir/project/pytorch-CycleGAN-and-pix2pix/results/all_stain_cyclegan/test_latest/images/"

for directory in os.listdir(path):
    for file in os.listdir(path + directory):
        #converted_file = converted_files + file[:-4] + "_fake_B.png"        
#        if(os.path.exists(converted_file)):
            img=Image.open(path + directory + "/" + file).convert("RGB")
            features=np.array(img, dtype=np.float32)/255.0
#           processed_img = Image.open(converted_file)
            #processed_feature=np.array(processed_img, dtype=np.float32)/255.0
            #X_processed.append(processed_feature)
            X.append(features)
            if directory in classes:
                class_id = classes[directory]
            else:
                class_id = len(classes.keys())
                classes[directory] = class_id
            Y.append(class_id)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
#X_processed_train, X_processed_test, y_processed_train, y_processed_test = train_test_split(X_processed, Y, test_size=0.33, random_state=42)

#X_processed_train = np.array(X_processed_train)
#X_processed_test = np.array(X_processed_test)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

NUM_CLASSES = len(classes.keys())

def inceptionv3_model_fn(features, labels, mode):
    # Load Inception-v3 model.
    module = hub.Module("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1")
    input_layer = adjust_image(features["x"])
    outputs = module(input_layer)

    logits = tf.layers.dense(inputs=outputs, units=NUM_CLASSES)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    accuracy = tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])
    tf.summary.scalar("my_accuracy", accuracy[1])

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": accuracy}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


with tf.Graph().as_default() as g:
        # create input functions for train and evaluate methods.
        train_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": X_train},
            y=y_train,
            batch_size=20,
            num_epochs=None,
            shuffle=True)
        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": X_test},
            y=y_test,
            num_epochs=1,
            shuffle=False)
                              
        # Create an estimator
        classifier = tf.estimator.Estimator(
            model_fn=inceptionv3_model_fn, model_dir="breast_tn_inception")

        # Set up logging for predictions
        tensors_to_log = {"probabilities": "softmax_tensor"}
        logging_hook = tf.train.LoggingTensorHook(
            tensors=tensors_to_log, every_n_iter=100)

        # Train network.
        classifier.train(
            input_fn=train_input_fn,
            max_steps=500)

        # Evaluate the model and print results.
        eval_results = classifier.evaluate(input_fn=eval_input_fn)
        print(eval_results)